# Load Dataset

In [3]:
from tqdm.auto import tqdm

In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/PMIndiaData/PMIndiaSum/malayalm-malayalm/train.csv")

: 

In [3]:
df.head()

,source_url,target_url,text,summary
0,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,"1,756 കോടി രൂപ ചെലവുവരുന്ന 2017-18 മുതല്‍ 2019...",നവീകരിച്ച &#8221; ഖേലോ ഇന്ത്യാ&#8221; (കളിക്കൂ...
1,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ടോക്കിയോയിൽ നടക്കുന്ന പാരാലിമ്പിക് ഗെയിംസിൽ ബാ...,പാരാലിമ്പിക് ഗെയിംസിൽ ബാഡ്മിന്റണിൽ വെള്ളി മെഡൽ...
2,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,സര്‍ ഛോട്ടു റാം ജിയുടെ ജന്മവാര്‍ഷിക ദിനത്തില്‍...,സര്‍ ഛോട്ടു റാം ജിയുടെ ജന്മവാര്‍ഷികദിനത്തില്‍ ...
3,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ഇന്ത്യൻ കൗൺസിൽ ഓഫ് മെഡിക്കൽ റിസർച്ചും ജർമ്മനിയ...,ഇന്ത്യൻ കൗൺസിൽ ഓഫ് മെഡിക്കൽ റിസർച്ചും ജർമ്മനി...
4,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ഭഗവാന്‍ ബിര്‍സ മുണ്ഡയുടെയും കോടിക്കണക്കിനു ജന്...,‘ജന്‍ജാതീയ ഗൗരവ് ദിവസത്തില്‍ രാജ്യത്തെ ഗിരിവർഗ...


In [4]:
df = df.rename(columns={'Text': 'text', 'Summary': 'summary'})

In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[['text', 'summary']])

In [6]:
texts = df['text'].tolist()
references = df['summary'].tolist()

In [7]:
texts = texts[:300]

# Compute Metrics

In [8]:
import evaluate

# Load metrics once
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(predictions, references, rouge_stemmer=True):
    """
    Computes ROUGE and BLEU scores for given predictions and references.
    
    Args:
        predictions (list): List of predicted texts.
        references (list): List of reference texts.
        rouge_stemmer (bool): Whether to use stemming for ROUGE.
    
    Returns:
        dict: ROUGE and BLEU scores rounded to four decimal places.
    """
    
    # Compute ROUGE scores
    rouge_result = rouge.compute(predictions=predictions, references=references, use_stemmer=rouge_stemmer)
    
    # Compute BLEU scores
    bleu_result = bleu.compute(
        predictions=predictions,
        references=[[r] for r in references]
    )
    
    return {
        "ROUGE-1": round(rouge_result['rouge1'], 4),
        "ROUGE-2": round(rouge_result['rouge2'], 4),
        "ROUGE-L": round(rouge_result['rougeL'], 4),
        "BLEU": round(bleu_result["bleu"], 4)
    }

2025-06-17 16:19:45.518517: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 16:19:45.533829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750173585.552795 3597431 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750173585.558676 3597431 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750173585.574037 3597431 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Print Results

In [9]:
def print_results(predictions, num_results = 5):
    for i in range(num_results):
        print(f"Input Text:\n{texts[i]}\n")
        print(f"Generated Summary:\n{predictions[i]}\n")
        print(f"Reference Summary:\n{references[i]}\n")
        print("="*80)

# Bart

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "ai4bharat/IndicBART"  # Malayalam IndicBART

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).eval()

In [11]:
bos_id = tokenizer.bos_token_id
eos_id = tokenizer.eos_token_id
pad_id = tokenizer.pad_token_id


In [12]:
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator


def malayalam_to_devanagari(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'ml', 'hi') + ' </s> <2hi>'
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'ml', 'hi') + ' </s> <2hi>' for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

def devanagari_to_malayalam(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'hi', 'ml')
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'hi', 'ml') for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

In [14]:
batch_size = 8  # smaller batch size to show progress in chunks
predictions = []

prompt_template = """
Generate a short and clear news headline in Malayalam based on the following text.

Text to generate headline for: {text}

The headline should capture the main idea of the text without copying it word-for-word.
Return the Headline in complete Malayalam.
You are not to return the thought process or any non Malayalam texts.
"""

for i in tqdm(range(0, len(texts), batch_size), desc="Generating summaries"):
    batch_texts = texts[i:i+batch_size]
    batch_texts = malayalam_to_devanagari(batch_texts)
    # inputs = tokenizer(batch_texts, max_length=512, truncation=True, padding=True, return_tensors='pt')
    # inputs = tokenizer("मैं [MASK] हूँ </s> <2hi>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids

    batch_prompts = [prompt_template.format(text=text) for text in batch_texts]
    inputs = tokenizer(batch_prompts, add_special_tokens=False, padding=True, return_tensors='pt')

    # summary_ids = model.generate(
    #     inputs['input_ids'], attention_mask=inputs['attention_mask'],
    #     max_length=128, num_beams=4, early_stopping=True
    # )

    summary_ids = model.generate(
        inputs['input_ids'], use_cache=True, num_beams=4, max_length=128,
        early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id,
        eos_token_id=eos_id,
        decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>")
        )

    batch_preds = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    predictions.extend(batch_preds)

predictions = devanagari_to_malayalam(predictions)

Generating summaries:   0%|          | 0/38 [00:00<?, ?it/s]

IndexError: index out of range in self

In [ ]:
compute_metrics(predictions, references[:16])

ROUGE-1 F1 Score: 0.05
ROUGE-2 F1 Score: 0.0417
ROUGE-L F1 Score: 0.05
BLEU Score: 0.0322

Sample Predictions:

Input Text:
ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സിക

In [ ]:
print_results(predictions)

# Qwen

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "Qwen/Qwen2-1.5B-Instruct"


device = "cuda:1" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", trust_remote_code=True).to(device).eval()

torch.cuda.empty_cache()

In [15]:
predictions=[]

for text in tqdm(texts, desc="Generating summaries"):
  try:
    prompt = f"""
      Generate a short and clear news headline in Malayalam based on the following text.

      Text to generate headline for: {text}

      The headline should capture the main idea of the text without copying it word-for-word.
      Return the headline in complete Malayalam.
      You are not to return the thought process or any non Malayalam texts.
      """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    with torch.no_grad():
      generated_ids = model.generate(
          model_inputs.input_ids,
          max_new_tokens=512
      )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # if "Summary:" in response:
    #   summary_clean = response.split("Summary:")[-1].strip()
    # else:
    #   summary_clean = response.strip()

    predictions.append(response)

    torch.cuda.empty_cache()

  except torch.cuda.OutOfMemoryError:
    print("⚠️ CUDA OOM – Skipping this input.")
    torch.cuda.empty_cache()
    predictions.append("[OOM ERROR – Skipped]")

Generating summaries:   0%|          | 0/300 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Token indices sequence length is longer than the specified maximum sequence length for this model (37529 > 32768). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (32768). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


⚠️ CUDA OOM – Skipping this input.


In [17]:
compute_metrics(predictions, references[:300])


{'ROUGE-1': np.float64(0.0511),
 'ROUGE-2': np.float64(0.0083),
 'ROUGE-L': np.float64(0.0517),
 'BLEU': 0.0512}

In [18]:
print_results(predictions)

Input Text:
1,756 കോടി രൂപ ചെലവുവരുന്ന 2017-18 മുതല്‍ 2019-20 വരേയ്ക്കുള്ള നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അദ്ധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭായോഗം അംഗീകാരം നല്‍കി. രാജ്യത്തെ കായികരംഗത്തിന് വളരെ നിര്‍ണ്ണായകമായ ഒരു തീരുമാനമാണിത്. കായികമേഖലയെ മുഖ്യധാരയിലെത്തിച്ച് അതിലൂടെ വ്യക്തിത്വ-സാമൂഹിക-സാമ്പത്തിക വികസനം എന്നതാണ് പദ്ധതി ലക്ഷ്യമാക്കുന്നത്. നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് കായികമേഖലയില്‍ ഗുണപരമായ മാറ്റമുണ്ടാക്കാനാകും ഉണ്ടാക്കാനാകും. പശ്ചാത്തല വികസനം, സാമൂഹിക കായികമേഖല, പ്രതിഭകളെ കണ്ടെത്തല്‍, മികവിന് വേണ്ടിയുള്ള പരിശീലനം, മത്സരഘടന, കായിക സമ്പദ്ഘടന തുടങ്ങി എല്ലാ മേഖലയിലും ഇതിന്റെ ഗണമുണ്ടാകും മുഖ്യ സവിശേഷതകള്‍ – ഇതുവരെയില്ലായിരുന്ന തരത്തിലുള്ള ഒരു വിശാല ഇന്ത്യാ കായിക സ്‌കോളര്‍ഷിപ്പ് പദ്ധതി ഇതിലൂടെ നടപ്പാക്കും. ഓരോ വര്‍ഷവും തെരഞ്ഞെടുക്കപ്പെടുന്ന കായികമേഖലയിലെ 1000 താരങ്ങള്‍ ഇതില്‍ ഉള്‍പ്പെടും. – ഈ പദ്ധതിയുടെ ഭാഗമായി തെരഞ്ഞെടുക്കപ്പെടുന്ന എല്ലാ കായികതാരങ്ങള്‍ക്കും അടുത്ത തുടര്‍ച്ചയായ എട്ടുവര്‍ഷത്തേക്ക് 

# Gemma



In [19]:
from huggingface_hub import login
TOKEN=None
login(token=TOKEN)

In [1]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch

model_id = "google/gemma-3-4b-it"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = Gemma3ForConditionalGeneration.from_pretrained(model_id).to(device).eval()

processor = AutoProcessor.from_pretrained(model_id, use_fast=True)
torch.set_float32_matmul_precision('high')

2025-06-21 13:16:13.842557: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-21 13:16:13.856911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750508173.873132   13052 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750508173.878484   13052 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750508173.891506   13052 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
predictions = []

for text in tqdm(texts, desc="Generating summaries"):
  try:
    prompt = f"""
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""

    messages = [
        [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant that generates news headlines."},]
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt},]
            },
        ],
    ]

    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
      generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
      generation = generation[0][input_len:]

    # Slice off input tokens from generated tokens to get only new tokens
    # generated_only_ids = [
    #     gen_ids[len(input_ids[idx]):] for idx, gen_ids in enumerate(generated_ids)
    # ]

    decoded = processor.decode(generation, skip_special_tokens=True)
    predictions.append(decoded)

    torch.cuda.empty_cache()

  except torch.cuda.OutOfMemoryError:
    print("⚠️ CUDA OOM – Skipping this input.")
    torch.cuda.empty_cache()
    predictions.append("[OOM ERROR – Skipped]")

NameError: name 'tqdm' is not defined

In [22]:
import re

predictions = [re.sub(r"(?i)^\s*Headline:\s*", "", p) for p in predictions]

In [23]:
compute_metrics(predictions, references[:300])

{'ROUGE-1': np.float64(0.0222),
 'ROUGE-2': np.float64(0.005),
 'ROUGE-L': np.float64(0.0228),
 'BLEU': 0.0407}

In [ ]:
print_results(predictions)

Input Text:
1,756 കോടി രൂപ ചെലവുവരുന്ന 2017-18 മുതല്‍ 2019-20 വരേയ്ക്കുള്ള നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അദ്ധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭായോഗം അംഗീകാരം നല്‍കി. രാജ്യത്തെ കായികരംഗത്തിന് വളരെ നിര്‍ണ്ണായകമായ ഒരു തീരുമാനമാണിത്. കായികമേഖലയെ മുഖ്യധാരയിലെത്തിച്ച് അതിലൂടെ വ്യക്തിത്വ-സാമൂഹിക-സാമ്പത്തിക വികസനം എന്നതാണ് പദ്ധതി ലക്ഷ്യമാക്കുന്നത്. നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് കായികമേഖലയില്‍ ഗുണപരമായ മാറ്റമുണ്ടാക്കാനാകും ഉണ്ടാക്കാനാകും. പശ്ചാത്തല വികസനം, സാമൂഹിക കായികമേഖല, പ്രതിഭകളെ കണ്ടെത്തല്‍, മികവിന് വേണ്ടിയുള്ള പരിശീലനം, മത്സരഘടന, കായിക സമ്പദ്ഘടന തുടങ്ങി എല്ലാ മേഖലയിലും ഇതിന്റെ ഗണമുണ്ടാകും മുഖ്യ സവിശേഷതകള്‍ – ഇതുവരെയില്ലായിരുന്ന തരത്തിലുള്ള ഒരു വിശാല ഇന്ത്യാ കായിക സ്‌കോളര്‍ഷിപ്പ് പദ്ധതി ഇതിലൂടെ നടപ്പാക്കും. ഓരോ വര്‍ഷവും തെരഞ്ഞെടുക്കപ്പെടുന്ന കായികമേഖലയിലെ 1000 താരങ്ങള്‍ ഇതില്‍ ഉള്‍പ്പെടും. – ഈ പദ്ധതിയുടെ ഭാഗമായി തെരഞ്ഞെടുക്കപ്പെടുന്ന എല്ലാ കായികതാരങ്ങള്‍ക്കും അടുത്ത തുടര്‍ച്ചയായ എട്ടുവര്‍ഷത്തേക്ക് 

: 

# By-T5-Small

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Load ByT5 and Flan-T5
byt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/byt5-small").to(device).eval()
byt5_tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

In [11]:
def generate_summary(text, model, tokenizer):
    prompt = f"""
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            num_beams=2,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()


In [ ]:
byt5_preds = []
for text in tqdm(texts, desc="ByT5-Small"):
    generated_summary = generate_summary(text, byt5_model, byt5_tokenizer)
    generated_summary = generate_summary(text, byt5_model, byt5_tokenizer)
    byt5_preds.append(generated_summary)


ByT5-Small:   0%|          | 0/300 [00:00<?, ?it/s]

ValueError: Length of values (300) does not match length of index (4433)

In [15]:
import re

# Remove any English word followed by a colon and text inside parentheses
byt5_preds = [re.sub(r"(?i)^\s*\w+:\s*|\s*\([^)]*\)|:\s*", "", p) for p in byt5_preds]


In [16]:
compute_metrics(byt5_preds, references[:300])

{'ROUGE-1': np.float64(0.0086),
 'ROUGE-2': np.float64(0.0011),
 'ROUGE-L': np.float64(0.0082),
 'BLEU': 0.0397}

In [17]:
print_results(byt5_preds)

Input Text:
1,756 കോടി രൂപ ചെലവുവരുന്ന 2017-18 മുതല്‍ 2019-20 വരേയ്ക്കുള്ള നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അദ്ധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭായോഗം അംഗീകാരം നല്‍കി. രാജ്യത്തെ കായികരംഗത്തിന് വളരെ നിര്‍ണ്ണായകമായ ഒരു തീരുമാനമാണിത്. കായികമേഖലയെ മുഖ്യധാരയിലെത്തിച്ച് അതിലൂടെ വ്യക്തിത്വ-സാമൂഹിക-സാമ്പത്തിക വികസനം എന്നതാണ് പദ്ധതി ലക്ഷ്യമാക്കുന്നത്. നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് കായികമേഖലയില്‍ ഗുണപരമായ മാറ്റമുണ്ടാക്കാനാകും ഉണ്ടാക്കാനാകും. പശ്ചാത്തല വികസനം, സാമൂഹിക കായികമേഖല, പ്രതിഭകളെ കണ്ടെത്തല്‍, മികവിന് വേണ്ടിയുള്ള പരിശീലനം, മത്സരഘടന, കായിക സമ്പദ്ഘടന തുടങ്ങി എല്ലാ മേഖലയിലും ഇതിന്റെ ഗണമുണ്ടാകും മുഖ്യ സവിശേഷതകള്‍ – ഇതുവരെയില്ലായിരുന്ന തരത്തിലുള്ള ഒരു വിശാല ഇന്ത്യാ കായിക സ്‌കോളര്‍ഷിപ്പ് പദ്ധതി ഇതിലൂടെ നടപ്പാക്കും. ഓരോ വര്‍ഷവും തെരഞ്ഞെടുക്കപ്പെടുന്ന കായികമേഖലയിലെ 1000 താരങ്ങള്‍ ഇതില്‍ ഉള്‍പ്പെടും. – ഈ പദ്ധതിയുടെ ഭാഗമായി തെരഞ്ഞെടുക്കപ്പെടുന്ന എല്ലാ കായികതാരങ്ങള്‍ക്കും അടുത്ത തുടര്‍ച്ചയായ എട്ടുവര്‍ഷത്തേക്ക് 

# **mT5**


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/umt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/umt5-base").eval()

In [ ]:
import torch
torch.cuda.empty_cache()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device);

In [19]:
prompt_template = """
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a one sentence news headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: headline:
"""



predictions = []
batch_size = 8  # Adjust depending on GPU memory

for i in tqdm(range(0, len(texts), batch_size), desc="Generating summaries"):
    batch_texts = texts[i:i+batch_size]

    prompts = [prompt_template.format(text=text) for text in batch_texts]

    # Tokenize inputs
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    
    ######################################################
    inputs.pop("token_type_ids", None) # used only with umt
    ######################################################

    # Generate summaries
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            early_stopping=True,
            num_beams=4,
            length_penalty=1.0,
            no_repeat_ngram_size=3,
        )

    # Decode outputs
    decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    decoded_batch = [d.replace("headline:", "").strip() for d in decoded_batch]
    predictions.extend(decoded_batch)

# Now predictions has your model summaries for all texts

Generating summaries:   0%|          | 0/38 [00:00<?, ?it/s]

In [20]:
compute_metrics(predictions,references[:300])

{'ROUGE-1': np.float64(0.0291),
 'ROUGE-2': np.float64(0.0023),
 'ROUGE-L': np.float64(0.0282),
 'BLEU': 0.0087}

In [21]:
print_results(predictions, 20)

Input Text:
1,756 കോടി രൂപ ചെലവുവരുന്ന 2017-18 മുതല്‍ 2019-20 വരേയ്ക്കുള്ള നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അദ്ധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭായോഗം അംഗീകാരം നല്‍കി. രാജ്യത്തെ കായികരംഗത്തിന് വളരെ നിര്‍ണ്ണായകമായ ഒരു തീരുമാനമാണിത്. കായികമേഖലയെ മുഖ്യധാരയിലെത്തിച്ച് അതിലൂടെ വ്യക്തിത്വ-സാമൂഹിക-സാമ്പത്തിക വികസനം എന്നതാണ് പദ്ധതി ലക്ഷ്യമാക്കുന്നത്. നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് കായികമേഖലയില്‍ ഗുണപരമായ മാറ്റമുണ്ടാക്കാനാകും ഉണ്ടാക്കാനാകും. പശ്ചാത്തല വികസനം, സാമൂഹിക കായികമേഖല, പ്രതിഭകളെ കണ്ടെത്തല്‍, മികവിന് വേണ്ടിയുള്ള പരിശീലനം, മത്സരഘടന, കായിക സമ്പദ്ഘടന തുടങ്ങി എല്ലാ മേഖലയിലും ഇതിന്റെ ഗണമുണ്ടാകും മുഖ്യ സവിശേഷതകള്‍ – ഇതുവരെയില്ലായിരുന്ന തരത്തിലുള്ള ഒരു വിശാല ഇന്ത്യാ കായിക സ്‌കോളര്‍ഷിപ്പ് പദ്ധതി ഇതിലൂടെ നടപ്പാക്കും. ഓരോ വര്‍ഷവും തെരഞ്ഞെടുക്കപ്പെടുന്ന കായികമേഖലയിലെ 1000 താരങ്ങള്‍ ഇതില്‍ ഉള്‍പ്പെടും. – ഈ പദ്ധതിയുടെ ഭാഗമായി തെരഞ്ഞെടുക്കപ്പെടുന്ന എല്ലാ കായികതാരങ്ങള്‍ക്കും അടുത്ത തുടര്‍ച്ചയായ എട്ടുവര്‍ഷത്തേക്ക് 

# Deepseek

In [10]:
 torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "deepseek-ai/deepseek-moe-16b-chat"

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True).to(device)
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
from types import MethodType

# Override `prepare_inputs_for_generation` to fix get_max_length -> get_seq_length
def patched_prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None, **kwargs):
    if past_key_values:
        input_ids = input_ids[:, -1:]
    return {
        "input_ids": input_ids,
        "past_key_values": past_key_values,
        "use_cache": kwargs.get("use_cache", True),
        "attention_mask": attention_mask,
    }

# Attach the patched method to your model
model.prepare_inputs_for_generation = MethodType(patched_prepare_inputs_for_generation, model)


In [12]:
# Prompt template
prompt_template = """
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a one sentence news headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: headline:
"""

# Inference loop
predictions =[]

for text in tqdm(texts, desc="Generating summaries"):
    prompt = prompt_template.format(text=text).strip()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
        )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the generated part (after the prompt)
    generated = output_text[len(prompt):].strip()
    generated = generated.replace("headline:", "").strip()
    
    predictions.append(generated)

    torch.cuda.empty_cache()


Generating summaries:   0%|          | 0/300 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Token indices sequence length is longer than the specified maximum s

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.79 GiB. GPU 1 has a total capacity of 79.25 GiB of which 1.96 GiB is free. Including non-PyTorch memory, this process has 48.83 GiB memory in use. Process 3581849 has 28.43 GiB memory in use. Of the allocated memory 41.41 GiB is allocated by PyTorch, and 6.93 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
compute_metrics(predictions,references[:300])

{'ROUGE-1': np.float64(0.0),
 'ROUGE-2': np.float64(0.0),
 'ROUGE-L': np.float64(0.0),
 'BLEU': 0.0064}

In [ ]:
print_results(predictions)

Input Text:
ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സികളുമായി എം.ഇ.ഐ.ടി.വൈ. കരാറുകളും ധാരണാപത്രങ്ങളും ഒപ്പുവെച്ചിട്ടുണ്ട്. ഡിജിറ്റല്‍ ഇന്ത്യ, മെയ്ക്ക് ഇന്‍ ഇന്ത്യ തുടങ

: 

# mBART

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MBartForConditionalGeneration, MBart50TokenizerFast

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").eval()

In [15]:
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [16]:
src_lang = "ml_IN"
tgt_lang = "ml_IN"

tokenizer.src_lang = src_lang
tokenizer.tgt_lang = tgt_lang


In [17]:
def generate_headlines(texts, max_length=64, num_beams=4):
    model.eval()
    preds = []

    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        generated_ids = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True
        )

        pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        preds.append(pred)

    return preds


In [21]:
predicted_headlines = generate_headlines(texts)


In [23]:
compute_metrics(predicted_headlines, references[:300])

{'ROUGE-1': np.float64(0.003),
 'ROUGE-2': np.float64(0.0002),
 'ROUGE-L': np.float64(0.0031),
 'BLEU': 0.0}

In [26]:
print_results(predicted_headlines,20)

Input Text:
1,756 കോടി രൂപ ചെലവുവരുന്ന 2017-18 മുതല്‍ 2019-20 വരേയ്ക്കുള്ള നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അദ്ധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭായോഗം അംഗീകാരം നല്‍കി. രാജ്യത്തെ കായികരംഗത്തിന് വളരെ നിര്‍ണ്ണായകമായ ഒരു തീരുമാനമാണിത്. കായികമേഖലയെ മുഖ്യധാരയിലെത്തിച്ച് അതിലൂടെ വ്യക്തിത്വ-സാമൂഹിക-സാമ്പത്തിക വികസനം എന്നതാണ് പദ്ധതി ലക്ഷ്യമാക്കുന്നത്. നവീകരിച്ച ” ഖേലോ ഇന്ത്യാ” (കളിക്കൂ ഇന്ത്യാ) പദ്ധതിക്ക് കായികമേഖലയില്‍ ഗുണപരമായ മാറ്റമുണ്ടാക്കാനാകും ഉണ്ടാക്കാനാകും. പശ്ചാത്തല വികസനം, സാമൂഹിക കായികമേഖല, പ്രതിഭകളെ കണ്ടെത്തല്‍, മികവിന് വേണ്ടിയുള്ള പരിശീലനം, മത്സരഘടന, കായിക സമ്പദ്ഘടന തുടങ്ങി എല്ലാ മേഖലയിലും ഇതിന്റെ ഗണമുണ്ടാകും മുഖ്യ സവിശേഷതകള്‍ – ഇതുവരെയില്ലായിരുന്ന തരത്തിലുള്ള ഒരു വിശാല ഇന്ത്യാ കായിക സ്‌കോളര്‍ഷിപ്പ് പദ്ധതി ഇതിലൂടെ നടപ്പാക്കും. ഓരോ വര്‍ഷവും തെരഞ്ഞെടുക്കപ്പെടുന്ന കായികമേഖലയിലെ 1000 താരങ്ങള്‍ ഇതില്‍ ഉള്‍പ്പെടും. – ഈ പദ്ധതിയുടെ ഭാഗമായി തെരഞ്ഞെടുക്കപ്പെടുന്ന എല്ലാ കായികതാരങ്ങള്‍ക്കും അടുത്ത തുടര്‍ച്ചയായ എട്ടുവര്‍ഷത്തേക്ക് 